In [1]:
import time
import threading

Threading allows us to run multiple tasks concurrently, so if I have two tasks, task A and task B, I can run them simultaneously without having to wait for task A to finish before running running task B.

When task run simultaneously, they're utilizing multiple CPUs. However, this can cause a problem in Python due to the GIL (Global Interpreter Lock). The GIL prevents threads from actually running in parallel and they can only use one processor at a time within a script, so to actually run tasks concurrently, Python uses another technique called task switching. Python rapidly switches between each thread to make it seem like our program is actually running multiple tasks in parallel.

This can actually be very useful for event-driven tasks that have a lot of downtime and are waiting for the user to input something.


Before we create/initialize a thread, we need to create a function.

In [2]:
def sleeper(n, name):
    print('I am {}. Going to sleep for {} seconds'.format(name, n))
    time.sleep(n)
    print('{} has woken up from sleep'.format(name))

We then initialize our thread with the `Thread` class from the `threading` module.

- `target`: accepts the function that we're going to execute
- `name`: is basically just naming the thread; this allows us to easily differentiate between threads when we have multiple threads
- `args`: pass in the argument to our function here

In [3]:
# we call .start to start executing the function from the thread
thread = threading.Thread(target = sleeper, name = 'thread1', args = (3, 'thread1'))
thread.start()

I am thread1. Going to sleep for 3 seconds
thread1 has woken up from sleep


Suppose we consider the main program as the main thread and our thread as its own separate thread, when we run a program and something is sleeping for a few seconds we have wait for thatportion to wake up before we can continue with the rest of the program. The code chunk below demonstrates the concurrency property, i.e. we don't have to wait for the thread we created to finish before running the rest of our program.

In [4]:
# hello is printed before the wake up message from the function
thread = threading.Thread(target = sleeper, name = 'thread1', args = (3, 'thread2'))
thread.start()

print()
print('hello')

I am thread2. Going to sleep for 3 seconds

hello
thread2 has woken up from sleep


Sometimes, we don't want Python to switch to the main thread until the thread we defined has finished executing its function. To do this, we can use `.join` method, this is essentially what they call a blocking call. It blocks the interpreter from accessing or executing the main program until the thread finishes it task.

In [5]:
# hello is printed after the wake up message from the function
thread = threading.Thread(target = sleeper, name = 'thread1', args = (3, 'thread2'))
thread.start()
thread.join()

print()
print('hello')

I am thread2. Going to sleep for 3 seconds
thread2 has woken up from sleep

hello


## Multithreading

The following code chunk showcase how to initialize and utilize multiple threads.

In [6]:
# create n_threads number of threads and store them in a list
n_threads = 5
threads = []

start = time.time()
for i in range(n_threads):
    name = 'thread_{}'.format(i)
    thread = threading.Thread(target = sleeper, name = name, args = (3, name))
    threads.append(thread)
    thread.start() 

for thread in threads:
    thread.join()

# we can see from the elapse time that it doesn't take
# n_threads * (the time we told the sleep function to sleep) amount
# of time to finish all the task
elapse = time.time() - start
print('Elapse time: ', elapse)
print('All {} tasks have finished their job'.format(n_threads))

I am thread_0. Going to sleep for 3 secondsI am thread_3. Going to sleep for 3 secondsI am thread_2. Going to sleep for 3 secondsI am thread_1. Going to sleep for 3 seconds



I am thread_4. Going to sleep for 3 seconds
thread_4 has woken up from sleep
thread_0 has woken up from sleepthread_1 has woken up from sleepthread_2 has woken up from sleepthread_3 has woken up from sleepElapse time:  


3.007716178894043

All 5 tasks have finished their job


The take home message is that threads are very efficient and beneficial for task that are not CPU intensive. Threads are really efficient at using any downtime or idle time, so when one thread is waiting for something we can have another thread performing another task. So in our example, when one thread is sleeping or not using much of the CPU resources, we can switch to execute another task to make sure the CPU is always busy.

Things like networking and web services are two type of operations that these small idle tiems that can be made more efficient using threads.

When we have something that's CPU intensive, there's actually something called multiprocessing.


## Daemon Threads

Daemon is an attribute for our threads. i.e. when we initialize the threads we have an option to set daemon as true or false. What daemon does is that it ends the thread when the main program finishes. 

# Reference

- [Youtube: Python Threading - Multithreading Playlist](https://www.youtube.com/playlist?list=PLGKQkV4guDKEv1DoK4LYdo2ZPLo6cyLbm)